### Procédure de test et fonctionnement de `incertitude.py`

**Contexte :** Ce fichier fournit des fonctions pour calculer l'abscisse de contact `xs` d'une particule dans un champ électrique uniforme et pour estimer l'incertitude sur `xs` due aux incertitudes sur les paramètres initiaux.

**Paramètres de base pour les tests (sauf indication contraire) :**
- `v0` = 2e5 m/s
- `theta` = π/6 radians (30°)
- `y0` = 0.05 m
- `q` = 1 * `constants.e` C (charge élémentaire positive)
- `m` = 1 * `constants.u` kg (masse d'1 unité atomique)
- `E` = -100000 V/m (calculé via `champ_electrique_v2(0.05, -5000)`)

**Incertitudes relatives pour les tests (sauf indication contraire) :**
- `delta_v0_rel` = 0.01 (1%)
- `delta_theta_rel` = 0.02 (2%)
- `delta_y0_rel` = 0.05 (5%)
- `delta_q_rel` = 0.001 (0.1%)
- `delta_m_rel` = 0.001 (0.1%)
- `delta_E_rel` = 0.03 (3%)

**Conversion en incertitudes absolues :** `delta_param = param * delta_param_rel`

**1. Test de la Fonction `champ_electrique_v2`**

| ID  | Fonctionnalité                 | Procédure                                               | Données de test                      | Résultat attendu                                                                       |
| :-- | :----------------------------- | :------------------------------------------------------ | :----------------------------------- | :------------------------------------------------------------------------------------- |
| 1| `champ_electrique_v2` valide   | Appeler `champ_electrique_v2(0.05, -5000)`              | distance=0.05 m, potentiel=-5000 V | Retourne -100000.0 V/m.                                                                |
| 2| `champ_electrique_v2` dist=0   | Tenter `champ_electrique_v2(0, -5000)`                  | distance=0 m, potentiel=-5000 V    | Lève une `ValueError` avec message "La distance doit être positive.".                 |
| 3| `champ_electrique_v2` dist<0   | Tenter `champ_electrique_v2(-0.05, -5000)`              | distance=-0.05 m, potentiel=-5000 V| Lève une `ValueError` avec message "La distance doit être positive.".                 |

**2. Test de la Fonction `calculer_xs`**

| ID  | Fonctionnalité            | Procédure                                                                           | Données de test (paramètres de base) | Résultat attendu                                                                                                |
| :-- | :------------------------ | :---------------------------------------------------------------------------------- | :----------------------------------- | :-------------------------------------------------------------------------------------------------------------- |
| 1| `calculer_xs` valide      | Appeler `calculer_xs(v0, theta, y0, q, m, E)` avec les paramètres de base.         | Paramètres de base                   | Retourne une valeur `xs` float positive (approx. 0.0203 m).                                                      |
| 2| `calculer_xs` pas contact | Appeler `calculer_xs(v0, theta, y0, q, m, E=1000)` (E positif, repousse la particule) | E = +1000 V/m                         | Lève une `ValueError` avec message "Le discriminant est négatif (...), pas de solution réelle pour xs (...)". |
| 3| `calculer_xs` E=0         | Appeler `calculer_xs(v0, theta, y0, q, m, E=0)`                                     | E = 0 V/m                             | Lève une `RuntimeWarning: divide by zero` (division par C), retourne `inf` ou `NaN` (selon version numpy).  *Note: formule non adaptée pour E=0.*|

**3. Test de la Fonction `derivees_partielles`**

| ID  | Fonctionnalité                 | Procédure                                                                               | Données de test (paramètres de base) | Résultat attendu                                                                                                                                                             |
| :-- | :----------------------------- | :-------------------------------------------------------------------------------------- | :----------------------------------- | :--------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| 1| `derivees_partielles` valide   | Appeler `derivees_partielles(v0, theta, y0, q, m, E)` avec les paramètres de base.    | Paramètres de base                   | Retourne un tuple de 6 floats. Les valeurs exactes nécessitent un calcul analytique de référence (e.g., `dxs_dy0` ≈ -0.235). Pas d'erreur.                                          |
| 2| `derivees_partielles` disc < 0 | Appeler `derivees_partielles(v0, theta, y0, q, m, E=1000)` (pas de contact)             | E = +1000 V/m                         | Lève `ValueError` (dans `np.sqrt` pour `D`) ou `RuntimeWarning: invalid value encountered in sqrt` et retourne des `NaN` si l'erreur n'est pas levée avant.                          |
| 3| `derivees_partielles` disc = 0 | (Difficile à atteindre exactement, mais si `A**2 - B` est très proche de 0)             | Valeurs spécifiques pour D ≈ 0       | Potentiellement `RuntimeWarning: divide by zero` (division par `D`) et retourne des `inf` ou `NaN`. *Note: la formule est instable si D=0.*                                   |

**4. Test de la Fonction `calculer_incertitude`**

| ID  | Fonctionnalité                   | Procédure                                                                                                                                    | Données de test (paramètres de base et incertitudes relatives/absolues) | Résultat attendu                                                                                                                             |
| :-- | :------------------------------- | :------------------------------------------------------------------------------------------------------------------------------------------- | :---------------------------------------------------------------------- | :------------------------------------------------------------------------------------------------------------------------------------------- |
| 1| `calculer_incertitude` valide  | Appeler `calculer_incertitude(v0, theta, y0, q, m, E, delta_v0, delta_theta, delta_y0, delta_q, delta_m, delta_E)` avec les valeurs calculées. | Paramètres de base & incertitudes abs. | Retourne une valeur `delta_xs` float positive (approx. 0.00115 m avec les deltas de `if __name__ == "__main__":`). Pas d'erreur.                |
| 2| `calculer_incertitude` une seule delta non nulle | `delta_v0_abs = v0 * delta_v0_rel`, autres deltas = 0. Appeler `calculer_incertitude(...)`.                                   | Seul `delta_v0` est non nul.           | Retourne `abs(dxs_dv0 * delta_v0_abs)`.                                                                                                       |
| 3| `calculer_incertitude` disc < 0| Tenter d'appeler avec `E=1000` (provoque erreur dans `derivees_partielles`).                                                                   | E = +1000 V/m                           | Lève une `ValueError` (ou `RuntimeWarning` propageant des `NaN`) hérité de `derivees_partielles`.                                              |

**5. Test du Bloc `if __name__ == "__main__":` (Tracé Graphique)**

| ID  | Fonctionnalité                                  | Procédure                                                                                                   | Données de test                                                | Résultat attendu                                                                                                                                    |
| :-- | :---------------------------------------------- | :---------------------------------------------------------------------------------------------------------- | :------------------------------------------------------------- | :-------------------------------------------------------------------------------------------------------------------------------------------------- |
| 1| Exécution du script principal                   | Exécuter le fichier `incertitude.py` directement (`python incertitude.py`).                                   | Paramètres définis dans le bloc `__main__`.                     | Le script s'exécute sans erreur. Une fenêtre Matplotlib s'affiche montrant un graphique de "Δ xs (m)" en fonction de "m/q (kg/C)". L'axe x est logarithmique. |
| 2| Cohérence des données du graphique (visuel)       | Examiner le graphique produit par I_M1.                                                                      | Paramètres du bloc `__main__`.                                  | L'incertitude `Δxs` devrait montrer une tendance (par exemple, augmenter ou diminuer avec m/q). Les valeurs sur les axes semblent raisonnables.            |

*Note :*
*   *Les valeurs numériques exactes pour les "Résultats attendus" peuvent varier légèrement en fonction des constantes physiques précises utilisées par `scipy.constants` et des arrondis.*
*   *Certains cas (comme E=0 pour `calculer_xs` ou D=0 pour `derivees_partielles`) mettent en évidence les limites des formules analytiques utilisées, qui peuvent ne pas être définies ou numériquement stables dans ces conditions extrêmes. Le code actuel ne gère pas explicitement ces cas par des `try-except` spécifiques dans les fonctions, mais s'appuie sur le comportement de NumPy (Warnings, NaN, inf).*
*   *Les tests I_D2, I_D3, I_CI3 sont importants pour comprendre comment les erreurs se propagent depuis les fonctions de bas niveau.*